In [1]:
%load_ext rpy2.ipython

In [4]:
n_int_AB = 5
n_int_ABC = 7
ILS = 30

In [10]:
%%R

library(tidyverse)

hid_tab <- read_csv('hid_tab.csv')
tree_tab <- read_csv('tree_tab.csv')
post_tab <- read_csv('post_tab.csv')
tree_tab_cut <- read_csv('tree_tab_cutpoints.csv')
post_tab_cut <- read_csv('post_tab_cutpoints.csv')

Rows: 119 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): name, topology, int_1, int_2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 155 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (7): gr, name, start, end, topology, int_1, int_2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 11900000 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (6): pos, name, value, topology, int_1, int_2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 143 Columns: 7
── Column specif

In [11]:
%%R -w 2000 -h 700 -r 150 -i n_int_AB -i n_int_ABC

p1 <- post_tab_cut %>%
    mutate(is_V0 = topology == 0) %>%
    group_by(pos, is_V0, int_1) %>%
    summarize(prob = sum(value)) %>%
    ggplot() +
    geom_tile(aes(pos, int_1+(!is_V0)*(n_int_AB), fill = prob, color = prob)) +
    geom_hline(aes(yintercept = n_int_AB-1+0.5), color = 'white') +
    geom_segment(aes(x = start, xend = end, y = int_1+(!(topology == 0))*(n_int_AB), yend = int_1+(!(topology == 0))*(n_int_AB)), 
                 color = 'green3', size = 2,
                 data = tree_tab_cut) +
    scale_fill_viridis_c(name = 'Posterior\nprobability', 
                         # limits = c(0, 1)
                         option="inferno"
                        ) +
    scale_color_viridis_c(name = 'Posterior\nprobability', 
                         # limits = c(0, 1)
                         option="inferno"
                         ) +
    scale_x_continuous(expand = c(0, 0)) +
    scale_y_continuous(
        breaks = c(0:(n_int_AB-1), ((n_int_AB):(n_int_AB+n_int_ABC-1))+0.1), 
        labels = c(0:(n_int_AB-1), 0:(n_int_ABC-1)),
        expand = c(0, 0)
    ) +
    labs(y = 'First coalescent', x = 'Position')

`summarise()` has grouped output by 'pos', 'is_V0'. You can override using the `.groups` argument.


In [12]:
%%R -w 2000 -h 700 -r 150

p2 <- post_tab %>%
    group_by(pos, int_2) %>%
    summarize(prob = sum(value)) %>%
    ggplot() +
    geom_tile(aes(pos, int_2, fill = prob, color = prob)) +
    geom_segment(aes(x = start, xend = end, y = int_2, yend = int_2), 
                 color = 'green3', size = 2,
                 data = tree_tab) +
    scale_fill_viridis_c(name = 'Posterior\nprobability', 
                         # limits = c(0, 1)
                         option="inferno") +
    scale_color_viridis_c(name = 'Posterior\nprobability', 
                         # limits = c(0, 1)
                         option="inferno") +
    scale_x_continuous(expand = c(0, 0)) +
    scale_y_continuous(
        breaks = 0:(n_int_ABC-1), 
        labels = 0:(n_int_ABC-1),
        expand = c(0, 0)
    ) +
    labs(y = 'Second coalescent') +
    theme(
        axis.title.x = element_blank(),
        axis.text.x = element_blank(),
        axis.ticks.x=element_blank()
    )

`summarise()` has grouped output by 'pos'. You can override using the `.groups` argument.


In [13]:
%%R -w 2000 -h 700 -r 150

p3 <- post_tab %>%
    group_by(pos, topology) %>%
    summarize(prob = sum(value)) %>%
    ggplot() +
    geom_tile(aes(pos, topology, fill = prob, color = prob)) +
    geom_segment(aes(x = start, xend = end, y = topology, yend = topology), 
                 color = 'green3', size = 2,
                 data = tree_tab) +
    scale_fill_viridis_c(name = 'Posterior\nprobability', 
                         # limits = c(0, 1)
                         option="inferno") +
    scale_color_viridis_c(name = 'Posterior\nprobability', 
                         # limits = c(0, 1)
                         option="inferno") +
    scale_x_continuous(expand = c(0, 0)) +
    scale_y_continuous(
        breaks = c(0, 1, 2, 3), 
        labels = c('V0', 'V1', 'V2', 'V3'),
        expand = c(0, 0)
    ) +
    labs(y = 'Topology') +
    theme(
        axis.title.x = element_blank(),
        axis.text.x = element_blank(),
        axis.ticks.x=element_blank()
    )

`summarise()` has grouped output by 'pos'. You can override using the `.groups` argument.


In [14]:
%%R -w 2000 -h 1000 -r 150 -i ILS

library(patchwork)

p3/p2/p1 + 
  plot_layout(heights = c(4, n_int_ABC, n_int_AB+n_int_ABC))

# ggsave(paste0('posterior_decoding_', round(ILS), '.pdf'), width = 14, height = 7)
ggsave(paste0('posterior_decoding_', n_int_AB, '_', n_int_ABC, '_', round(ILS), '_combined.png'), width = 14, height = 9)